In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
import re
import json


DATA_DIR = 'data/'
WIKIPEDIA_BASE_URL = 'https://fr.wikipedia.org'

In [2]:
def parse_url(url):
    page = urllib.request.urlopen(url).read().decode('utf-8')
    return BeautifulSoup(page, 'html.parser')

def get_table_from_article(url_or_soup, table_index, col_indices, col_names, col_links_indices, col_links_names,
                           col_coords_indices=None, col_coords_names=None):
    soup = parse_url(url_or_soup) if isinstance(url_or_soup, str) else url_or_soup
    table = soup.find_all('table', class_='sortable')[table_index]
    col_coords_indices = col_coords_indices if col_coords_indices else []
    col_coords_names = col_coords_names if col_coords_names else []
    data = []
    for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        cols = [td.get_text(',', strip=True) for td in tds]
        cols = [cols[i] for i in col_indices]
        links = []
        for index in col_links_indices:
            link = tds[index].find('a', href=True)
            if link:
                link = link['href']
                if link.endswith('&action=edit&redlink=1'):
                    link = None
                elif link.startswith('/'):
                    link = WIKIPEDIA_BASE_URL + link
            else:
                link = None
            links.append(link)
        coords = []
        for index in col_coords_indices:
            coord = tds[index].find('a')
            if coord and coord.has_attr('data-lat'):
                coord = (float(coord['data-lat']), float(coord['data-lon']))
            else:
                coord = None
            coords.append(coord)
        data.append((*cols, *links, *coords))
    df = pd.DataFrame(data, columns=[*col_names, *col_links_names, *col_coords_names])
    return df

def extract_monuments_list(article_or_url, col_indices=None, col_coords_indices=None):
    col_indices = [0, 4, 5] if not col_indices else col_indices
    col_coords_indices = [2] if not col_coords_indices else col_coords_indices
    df2 = get_table_from_article(article_or_url, table_index=0, col_indices=col_indices,
                                 col_names=['Name', 'Status', 'Date'],
                                 col_links_indices=[0], col_links_names=['Link'],
                                 col_coords_indices=[2], col_coords_names=['Coords'])
    return df2[(df2.Status.str.contains('Classé')) & (df2.Link)]

In [3]:
url = 'https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_par_commune_fran%C3%A7aise'
df = get_table_from_article(url, table_index=1, col_indices=[1, 4], col_names=['Commune', 'Number of monuments'],
                            col_links_indices=[1], col_links_names=['Link'])
df.head(3)

,Commune,Number of monuments,Link
0,Paris,1823,https://fr.wikipedia.org/wiki/Liste_des_monume...
1,Bordeaux,365,https://fr.wikipedia.org/wiki/Liste_des_monume...
2,La Rochelle,292,https://fr.wikipedia.org/wiki/Monuments_de_La_...


In [4]:
monuments_per_commune = {}
for _, row in df.iterrows():
    article = parse_url(row.Link)
    keywords = ['Liste', 'Monuments historiques', 'Immobiliers', 'Immobilier', 'Monuments actuels',
                'Monuments immobiliers', 'Liste exhaustive', 'Caen']
    found_keywords = [article.find('span', class_='mw-headline', text=keyword) for keyword in keywords]
    if sum(map(bool, found_keywords)):
        monuments_per_commune[row.Commune] = extract_monuments_list(article)
    else:
        print('Skipping {} (<{}>)'.format(row.Commune, row.Link))

Skipping Paris (<https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_de_Paris>)
Skipping La Rochelle (<https://fr.wikipedia.org/wiki/Monuments_de_La_Rochelle>)


In [5]:
monuments_per_commune['Bordeaux'].head(3)

,Name,Status,Date,Link,Coords
0,Abbatiale Sainte-Croix,Classé,1840,https://fr.wikipedia.org/wiki/Abbatiale_Sainte...,"(44.831051, -0.560909)"
1,Ancien Hôtel de ville de Bordeaux,Classé,1886,https://fr.wikipedia.org/wiki/Grosse_cloche_de...,"(44.835436, -0.571333)"
2,Basilique Saint-Michel,Classé,1846,https://fr.wikipedia.org/wiki/Basilique_Saint-...,"(44.834438, -0.56549)"


In [6]:
urls = ['https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_du_1er_arrondissement_de_Paris',
        'https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_du_2e_arrondissement_de_Paris',
        'https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_du_3e_arrondissement_de_Paris',
        'https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_du_4e_arrondissement_de_Paris',
        'https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_du_5e_arrondissement_de_Paris']
paris_monuments = []
for i, url in enumerate(urls):
    paris_monuments.append(extract_monuments_list(url))
monuments_per_commune['Paris'] = pd.concat(paris_monuments)

In [7]:
monuments_per_commune['Paris'].head(3)

,Name,Status,Date,Link,Coords
0,"Ancien appartement de,Coco Chanel",Classé,"2013,[,1,]",https://fr.wikipedia.org/wiki/Coco_Chanel,"(48.8682, 2.3266)"
2,Bourse de commerce,Classé,1862,https://fr.wikipedia.org/wiki/Bourse_de_commer...,"(48.862833, 2.342778)"
8,Colonne Médicis,Classé,1862,https://fr.wikipedia.org/wiki/Colonne_M%C3%A9d...,"(48.862528, 2.343)"


In [8]:
url = 'https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_de_La_Rochelle'
monuments_per_commune['La Rochelle'] = extract_monuments_list(url)

In [9]:
url = 'https://fr.wikipedia.org/wiki/Liste_des_monuments_historiques_de_Lyon'
monuments_per_commune['Lyon'] = extract_monuments_list(url, col_indices=[0, 5, 6], col_coords_indices=[3])

In [10]:
monuments = pd.concat({k: v.reset_index(drop=True) for k, v in monuments_per_commune.items()}, sort=False)
monuments.loc['Abbeville']

,Name,Status,Date,Link,Coords
0,Carrière Carpentier,Classé,1983,https://fr.wikipedia.org/wiki/Carri%C3%A8re_Ca...,"(50.106164, 1.848031)"
1,Carrière de Menchecourt,Classé,1983,https://fr.wikipedia.org/wiki/Carri%C3%A8re_de...,"(50.118391, 1.823725)"
2,Église Notre-Dame-de-la-Chapelle,Classé,1910,https://fr.wikipedia.org/wiki/%C3%89glise_Notr...,"(50.117192, 1.831911)"
3,Église Saint-Sépulcre,Classé,1907,https://fr.wikipedia.org/wiki/%C3%89glise_Sain...,"(50.10914, 1.83728)"
4,Église Saint-Vulfran,Classé,1840,https://fr.wikipedia.org/wiki/%C3%89glise_Sain...,"(50.10472, 1.83194)"
5,Manufacture des Rames,"Inscrit,Classé","1984,1986",https://fr.wikipedia.org/wiki/Manufacture_des_...,"(50.110166, 1.822223)"


In [23]:
monuments["Link"][5]

'https://fr.wikipedia.org/wiki/Manufacture_des_Rames'

In [11]:
monuments.to_csv(DATA_DIR + '_monuments.csv')

In [12]:
df3 = pd.read_csv(DATA_DIR + '_stations.csv')
communes = [v.replace('Saint-', 'St-') for v in monuments.index.levels[0].values]
unknowns1 = set(communes) - set(df3[df3.Name.isin(communes)].Name)
print(len(unknowns1))
unknowns = unknowns1 - set(df3[df3.Commune.isin(communes)].Commune)
print(len(unknowns))
unknowns

62
26


{'Auvillar',
 'Billom',
 'Carnac',
 'Cayenne',
 'Collonges-la-Rouge',
 'Cordes-sur-Ciel',
 'Crémieu',
 'Fontvieille',
 'Fort-de-France',
 'Kaysersberg',
 'Le Faou',
 'Le Mont-St-Michel',
 'Les Baux-de-Provence',
 'Locronan',
 'Lyon',
 'Marseille',
 'Mirepoix',
 'Monpazier',
 'Paris',
 'Pérouges',
 'Richelieu',
 'Riquewihr',
 'St-Pierre',
 'Tréguier',
 'Villefranche-de-Conflent',
 "Île-d'Aix"}

In [13]:
monuments_per_commune['Crémieu'] = pd.DataFrame.from_dict([{'Name': '', 'Status': '', 'Date': '', 'Link': ':-('}])
manual_aliases = {'Auvillar': 'Valence-d\'Agen',
                  'Billom': 'Clermont-Ferrand',
                  'Carnac': 'Auray ',
                  'Collonges-la-Rouge': 'Brive-la-Gaillarde',
                  'Cordes-sur-Ciel': 'Cordes-Vindrac',
                  'Crémieu': 'Lyon-St-Exupéry-TGV',
                  'Fontvieille': 'Arles',
                  'Kaysersberg': 'Colmar',
                  'Le Faou': 'Brest',
                  'Le Mont-St-Michel': 'Pontorson-Mont-St-Michel',
                  'Les Baux-de-Provence': 'Tarascon',
                  'Locronan': 'Quimper',
                  'Lyon': 'Lyon-Part-Dieu',
                  'Marseille': 'Marseille-St-Charles',
                  'Mirepoix': 'Pamiers',
                  'Monpazier': 'Lalinde',
                  'Paris': 'Paris-Nord',
                  'Pérouges': 'Meximieux-Pérouges',
                  'Richelieu': 'Chinon',
                  'Riquewihr': 'Colmar',
                  'Tréguier': 'Paimpol',
                  'Villefranche-de-Conflent': 'Perpignan',
                  'Île-d\'Aix': 'La Rochelle-Ville',
                  'Lille': 'Lille-Europe',
                  'Bordeaux': 'Bordeaux-St-Jean',
                  'Auxerre': 'Auxerre-St-Gervais',
                  'Rouen': 'Rouen-Rive-Droite',
                  'Besançon': 'Besançon-Viotte',
                  'Avignon': 'Avignon-Centre',
                  'Angers': 'Angers-St-Laud',
                  'Chambéry': 'Chambéry-Challes-les-Eaux',
                  'Saumur': 'Saumur-Rive-Droite',
                  'St-Flour': 'St-Flour-Chaudes-Aigues',
                  'Mâcon': 'Mâcon-Ville',
                  'Nancy': 'Nancy-Ville',
                  'Toulouse': 'Toulouse-Matabiau',
                  'Pernes-les-Fontaines': 'Pernes',
                  'Montauban': 'Montauban-Ville-Bourbon',
                  'Dole': 'Dole-Ville',
                  'Limoges': 'Limoges-Bénédictins',
                  'Montluçon': 'Montluçon-Ville',
                  'Le Touquet-Paris-Plage': 'Le Touquet-Aéroport',
                  'Montpellier': 'Montpellier-St-Roch',
                  'Metz': 'Metz-Ville',
                  'St-Étienne': 'St-Étienne-Châteaucreux',
                  'Riom': 'Riom-Châtel-Guyon',
                  'Nice': 'Nice-Ville',
                  'Moulins': 'Moulins-sur-Allier',
                  'St-Germain-en-Laye': 'St-Germain-Grande-Ceinture',
                  'Fontainebleau': 'Fontainebleau-Avon',
                  'Rocamadour': 'Rocamadour-Padirac',
                  'Strasbourg': 'Strasbourg-Ville',
                  'Sarlat-la-Canéda': 'Sarlat',
                  'La Rochelle': 'La Rochelle-Ville',
                  'Dijon': 'Dijon-Ville',
                  'Les Eyzies-de-Tayac-Sireuil': 'Les Eyzies',
                  'Versailles': 'Versailles-Chantiers',
                  'Mulhouse': 'Mulhouse-Ville'}
# Cayenne, Fort-de-France, St-Pierre -> outre-mer, ignore
ignore = ['St-Paul'] # outre-mer (but also a commune in mainland France)
aliases_historic_cities = {}
for commune in set(communes):
    if commune in ignore:
        print('Ignoring {} (outre-mer)'.format(commune))
        continue
    elif commune in manual_aliases:
        nearest = manual_aliases[commune]
    elif commune in df3.Name.values:
        nearest = commune
    else:
        print('Ignoring {} (outre-mer)'.format(commune))
        continue
    aliases_historic_cities[nearest] = ([*aliases_historic_cities[nearest], commune]
                                        if nearest in aliases_historic_cities else [commune])

Ignoring St-Paul (outre-mer)
Ignoring St-Pierre (outre-mer)
Ignoring Cayenne (outre-mer)
Ignoring Fort-de-France (outre-mer)


In [14]:
aliases_historic_cities # Important stations that link to important cities

{'Abbeville': ['Abbeville'],
 'Agen': ['Agen'],
 'Aire-sur-la-Lys': ['Aire-sur-la-Lys'],
 'Aix-en-Provence': ['Aix-en-Provence'],
 'Ajaccio': ['Ajaccio'],
 'Albi': ['Albi'],
 'Alençon': ['Alençon'],
 'Alet-les-Bains': ['Alet-les-Bains'],
 'Amboise': ['Amboise'],
 'Amiens': ['Amiens'],
 'Angers-St-Laud': ['Angers'],
 'Angoulême': ['Angoulême'],
 'Annecy': ['Annecy'],
 'Arles': ['Arles', 'Fontvieille'],
 'Arras': ['Arras'],
 'Auch': ['Auch'],
 'Auray': ['Auray'],
 'Auray ': ['Carnac'],
 'Aurillac': ['Aurillac'],
 'Autun': ['Autun'],
 'Auxerre-St-Gervais': ['Auxerre'],
 'Avignon-Centre': ['Avignon'],
 'Bar-le-Duc': ['Bar-le-Duc'],
 'Bayeux': ['Bayeux'],
 'Bayonne': ['Bayonne'],
 'Beaucaire': ['Beaucaire'],
 'Beaune': ['Beaune'],
 'Beauvais': ['Beauvais'],
 'Besançon-Viotte': ['Besançon'],
 'Blois': ['Blois'],
 'Bordeaux-St-Jean': ['Bordeaux'],
 'Bourg-St-Andéol': ['Bourg-St-Andéol'],
 'Bourg-en-Bresse': ['Bourg-en-Bresse'],
 'Bourges': ['Bourges'],
 'Brest': ['Le Faou'],
 'Briançon': ['Br

In [15]:
url = 'https://fr.wikipedia.org/wiki/Villes_et_Pays_d%27art_et_d%27histoire'
df4 = get_table_from_article(url, table_index=0, col_indices=[0, 2], col_names=['Name', 'Number of communes'],
                            col_links_indices=[0], col_links_names=['Link'])
df4.loc[df4['Number of communes'] == '', 'Number of communes'] = '99'
df4['Number of communes'] = df4['Number of communes'].apply(lambda x: int(re.sub(',\[,Note.*', '', x)))
df4 = df4[(df4['Number of communes'] == 1) & (df4.Link)]

In [16]:
communes2 = df4.Name
communes2 = communes2.str.replace('Saint-', 'St-')

In [17]:
manual_aliases.update({
    'Aix-les-Bains': 'Aix-les-Bains-Le Revard',
    'Lodève': 'Montpellier-St-Roch',
    'Boulogne-sur-Mer': 'Boulogne-Tintelleries',
    'Rouen Métropole': 'Rouen-Rive-Droite',
    'Noisiel': 'Paris-Nord',
    'Boulogne-Billancourt': 'Paris-Nord',
    'Chantilly': 'Chantilly-Gouvieux',
    'Vincennes': 'Paris-Nord',
    'La Charité-sur-Loire': 'La Charité',
    
})
ignore.extend(['Pointe-à-Pitre', 'Parc naturel régional du Vexin français', 'Basse-Terre', 'Sartène',
               'St-Laurent-du-Maroni', 'St-Pierre'])
aliases_pays_dart_histoire = {}
for commune in set(communes2):
    if commune in ignore:
        print('Ignoring {} (outre-mer)'.format(commune))
        continue
    elif commune in manual_aliases:
        nearest = manual_aliases[commune]
    elif commune in df3.Name.values:
        nearest = commune
    else:
        print('Ignoring {} (?)'.format(commune))
        continue
    aliases_pays_dart_histoire[nearest] = ([*aliases_pays_dart_histoire[nearest], commune]
                                           if nearest in aliases_pays_dart_histoire else [commune])

Ignoring Sartène (outre-mer)
Ignoring St-Paul (outre-mer)
Ignoring St-Pierre (outre-mer)
Ignoring St-Laurent-du-Maroni (outre-mer)
Ignoring Basse-Terre (outre-mer)
Ignoring Pointe-à-Pitre (outre-mer)
Ignoring Parc naturel régional du Vexin français (outre-mer)


In [18]:
aliases_pays_dart_histoire

{'Aix-les-Bains-Le Revard': ['Aix-les-Bains'],
 'Ajaccio': ['Ajaccio'],
 'Albertville': ['Albertville'],
 'Angers-St-Laud': ['Angers'],
 'Arles': ['Arles'],
 'Autun': ['Autun'],
 'Auxerre-St-Gervais': ['Auxerre'],
 'Bar-le-Duc': ['Bar-le-Duc'],
 'Bastia': ['Bastia'],
 'Bayonne': ['Bayonne'],
 'Beaucaire': ['Beaucaire'],
 'Beauvais': ['Beauvais'],
 'Belfort': ['Belfort'],
 'Bergerac': ['Bergerac'],
 'Bernay': ['Bernay'],
 'Besançon-Viotte': ['Besançon'],
 'Blois': ['Blois'],
 'Bordeaux-St-Jean': ['Bordeaux'],
 'Boulogne-Tintelleries': ['Boulogne-sur-Mer'],
 'Bourges': ['Bourges'],
 'Brest': ['Brest'],
 'Briançon': ['Briançon'],
 'Caen': ['Caen'],
 'Cahors': ['Cahors'],
 'Cambrai': ['Cambrai'],
 'Carcassonne': ['Carcassonne'],
 'Chalon-sur-Saône': ['Chalon-sur-Saône'],
 'Chambéry-Challes-les-Eaux': ['Chambéry'],
 'Chantilly-Gouvieux': ['Chantilly'],
 'Charleville-Mézières': ['Charleville-Mézières'],
 'Chinon': ['Chinon'],
 'Châlons-en-Champagne': ['Châlons-en-Champagne'],
 'Cognac': ['Co

In [19]:
with open(DATA_DIR + '_historic_cities.json', 'w') as outfile:
    json.dump(aliases_historic_cities, outfile, sort_keys=True)
with open(DATA_DIR + '_art_history_cities.json', 'w') as outfile:
    json.dump(aliases_pays_dart_histoire, outfile, sort_keys=True)